In [1]:
import pandas as pd
import numpy as np

In [2]:
gameInput = pd.read_csv("nhl-game-data/game.csv").dropna()
goalieInput = pd.read_csv("nhl-game-data/game_goalie_stats.csv")
skaterStatsInput = pd.read_csv("nhl-game-data/game_skater_stats.csv")
teamsStatsInput = pd.read_csv("nhl-game-data/game_teams_stats.csv")
teamInfoInput = pd.read_csv("nhl-game-data/team_info.csv")

In [4]:
gameInput

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2011030221,20112012,P,2012-04-29,2012-04-29T19:00:00Z,1,4,3,4,home win OT,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
1,2011030222,20112012,P,2012-05-01,2012-05-01T23:30:00Z,1,4,4,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2011030223,20112012,P,2012-05-03,2012-05-03T23:30:00Z,4,1,3,4,home win OT,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
3,2011030224,20112012,P,2012-05-06,2012-05-06T23:30:00Z,4,1,2,4,home win REG,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
4,2011030225,20112012,P,2012-05-08,2012-05-08T23:30:00Z,1,4,3,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,P,2019-06-02,2019-06-02T00:00:00Z,6,19,7,2,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11430,2018030414,20182019,P,2019-06-04,2019-06-04T00:00:00Z,6,19,2,4,home win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11431,2018030415,20182019,P,2019-06-07,2019-06-07T00:00:00Z,19,6,2,1,away win REG,left,TD Garden,/api/v1/venues/5085,America/New_York,-4,EDT
11432,2018030416,20182019,P,2019-06-10,2019-06-10T00:00:00Z,6,19,5,1,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT


In [14]:
# gameAwayValues = gameInput.loc[:,['away_team_id','date_time', 'away_goals','home_goals','season']] 
gameAwayValues = gameInput.loc[:,['away_team_id', 'date_time', 'away_goals', 'home_goals', 'season']].groupby(['away_team_id', 'season']) \
       .agg({'date_time':'size', 'away_goals':'mean', 'home_goals':'mean'}) \
       .rename(columns={'date_time':'count'}) \
       .reset_index()
gameHomeValues = gameInput.loc[:,['home_team_id', 'date_time', 'away_goals', 'home_goals', 'season']].groupby(['home_team_id', 'season']) \
       .agg({'date_time':'size', 'away_goals':'mean', 'home_goals':'mean'}) \
       .rename(columns={'date_time':'count'}) \
       .reset_index()
gameAwayValues.head(5)

,away_team_id,season,count,away_goals,home_goals
0,1,20102011,39,1.948718,2.538462
1,1,20112012,53,2.566038,2.528302
2,1,20122013,24,1.916667,2.916667
3,1,20132014,40,2.250000,2.825000
4,1,20142015,41,2.195122,3.073171


In [41]:
homeTeamStats = pd.DataFrame(columns=['home_team_id', 'season', 'scored', 'conceded'])
awayTeamStats = pd.DataFrame(columns=['away_team_id', 'season', 'scored', 'conceded'])
for index, row in gameAwayValues.iterrows():
    if not gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['away_team_id']) & (gameHomeValues['season'] == row['season'])].empty:
        season = row['season']
        homeTeam = gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['away_team_id']) & (gameHomeValues['season'] == row['season'])]
        homeScored = (homeTeam['home_goals'].values[0]*homeTeam['count'].values[0] + row['home_goals']*row['count'])/(homeTeam['count'].values[0] + row['count'])
        awayScored = (homeTeam['away_goals'].values[0]*homeTeam['count'].values[0] + row['away_goals']*row['count'])/(homeTeam['count'].values[0] + row['count'])
        d1 = {'away_team_id': row['away_team_id'], 'season': season, 'conceded': awayScored, 'scored': homeScored}
        d2 = {'home_team_id': homeTeam['home_team_id'].values[0], 'season': season, 'conceded': awayScored, 'scored': homeScored}
        homeTeamStats.at[index, :] = d2
        awayTeamStats.at[index, :] = d1
awayTeamStats.head()

,away_team_id,season,scored,conceded
0,1,2.0102e+07,2.4125,2.2125
1,1,2.0112e+07,2.68571,2.51429
2,1,2.0122e+07,2.83333,2.1875
3,1,2.0132e+07,2.69136,2.24691
4,1,2.0142e+07,2.64634,2.19512


In [42]:
dataSet1 = pd.DataFrame(columns=['game_id', 'season', 'date_time', 'home_team_id', '1_scored', '1_conceded', 'away_team_id', '2_scored', '2_conceded'])
# print(gameAwayValues.loc[(gameAwayValues['away_team_id'] == '4') & (gameAwayValues['season'] == '20102011')])
for index, row in gameInput.iterrows():
    season = row['season'] - 10001
    if not awayTeamStats.loc[(awayTeamStats['away_team_id'] == row['away_team_id']) & (awayTeamStats['season'] == season)].empty:
        if not homeTeamStats.loc[(homeTeamStats['home_team_id'] == row['home_team_id']) & (homeTeamStats['season'] == season)].empty:
            awayTeam = awayTeamStats.loc[(awayTeamStats['away_team_id'] == row['away_team_id']) & (awayTeamStats['season'] == row['season'])]
            homeTeam = homeTeamStats.loc[(homeTeamStats['home_team_id'] == row['home_team_id']) & (homeTeamStats['season'] == row['season'])]
            d = {'game_id': row['game_id'], 'season': row['season'], 'date_time': row['date_time'], 'home_team_id': row['home_team_id'], '1_scored': homeTeam['scored'].values[0], '1_conceded': homeTeam['conceded'].values[0], 'away_team_id': row['away_team_id'], '2_scored': awayTeam['scored'].values[0], '2_conceded': awayTeam['conceded'].values[0]}
            dataSet1.at[index, :] = d
    

In [43]:
dataSet1

,game_id,season,date_time,home_team_id,1_scored,1_conceded,away_team_id,2_scored,2_conceded
0,2011030221,20112012,2012-04-29,4,3.1413,3.1087,1,2.68571,2.51429
1,2011030222,20112012,2012-05-01,4,3.1413,3.1087,1,2.68571,2.51429
2,2011030223,20112012,2012-05-03,1,2.68571,2.51429,4,3.1413,3.1087
3,2011030224,20112012,2012-05-06,1,2.68571,2.51429,4,3.1413,3.1087
4,2011030225,20112012,2012-05-08,4,3.1413,3.1087,1,2.68571,2.51429
...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,2019-06-02,19,2.78505,2.91589,6,2.97143,2.73333
11430,2018030414,20182019,2019-06-04,19,2.78505,2.91589,6,2.97143,2.73333
11431,2018030415,20182019,2019-06-07,6,2.97143,2.73333,19,2.78505,2.91589
11432,2018030416,20182019,2019-06-10,19,2.78505,2.91589,6,2.97143,2.73333


In [44]:
dataSet1.to_csv('.\datasets\dataset1.csv', encoding='utf-8', index=False)